In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# ==========================================
# 1. DATA PROCESSING PIPELINE
# ==========================================
class AccelPipeline:
    def __init__(self, df):
        self.df = df.copy()
        
        # Ensure Timestamp format
        self.df['local_ts'] = pd.to_datetime(self.df['local_ts'])
        
        # Sort by Subject THEN Time to ensure rolling window correctness
        self.df = self.df.sort_values(by=['subject', 'local_ts']).reset_index(drop=True)
        
    def convert_to_gravity(self):
        print("--- Converting to Gravity Units & Calculating ENMO ---")
        scale = 16384.0
        
        self.df['x_g'] = self.df['x'] / scale
        self.df['y_g'] = self.df['y'] / scale
        self.df['z_g'] = self.df['z'] / scale
        
        # Magnitude
        self.df['mag'] = np.sqrt(self.df['x_g']**2 + self.df['y_g']**2 + self.df['z_g']**2)
        
        # ENMO: max(mag - 1, 0)
        self.df['enmo'] = np.maximum(self.df['mag'] - 1, 0)
        return self.df

    def _get_dynamic_component(self, window_seconds=1):
        # Create temp df indexed by time
        temp_df = self.df.set_index('local_ts').sort_index()
        cols = ['x_g', 'y_g', 'z_g']
        
        # Group by subject -> Rolling Mean (Static Gravity)
        static_component = temp_df.groupby('subject')[cols].rolling(f'{window_seconds}s').mean()
        
        # Merge static values back to main dataframe
        static_reset = static_component.reset_index()
        merged = pd.merge(self.df, static_reset, on=['subject', 'local_ts'], suffixes=('', '_static'))
        
        # Dynamic = Raw - Static
        dynamic_df = pd.DataFrame()
        dynamic_df['x_d'] = merged['x_g'] - merged['x_g_static']
        dynamic_df['y_d'] = merged['y_g'] - merged['y_g_static']
        dynamic_df['z_d'] = merged['z_g'] - merged['z_g_static']
        
        return dynamic_df.fillna(0)

    def calc_odba(self):
        print("--- Calculating ODBA ---")
        dyn = self._get_dynamic_component()
        self.df['odba'] = dyn['x_d'].abs() + dyn['y_d'].abs() + dyn['z_d'].abs()
        return self.df

    def calc_vedba(self):
        print("--- Calculating VeDBA ---")
        dyn = self._get_dynamic_component()
        self.df['vedba'] = np.sqrt(dyn['x_d']**2 + dyn['y_d']**2 + dyn['z_d']**2)
        return self.df

    def resample_data(self, interval_seconds=5):
        print(f"--- Resampling data to {interval_seconds} second windows ---")
        
        # Aggregation Dictionary
        agg_dict = {
            'x_g': 'mean', 'y_g': 'mean', 'z_g': 'mean',
            'mag': 'mean', 'enmo': 'mean',
            'odba': 'mean', 'vedba': 'mean',
            # UPDATED: Mode for behavioral_category
            'behavioral_category': lambda x: x.mode()[0] if not x.mode().empty else np.nan
        }
        
        resampled_df = (
            self.df.set_index('local_ts')
            .groupby('subject')
            .resample(f'{interval_seconds}s')
            .agg(agg_dict)
        )
        
        return resampled_df.dropna().reset_index()

# ==========================================
# 2. MACHINE LEARNING MODELER
# ==========================================



# 1. Load Data


In [18]:
import pandas as pd
import numpy as np
import optuna
import xgboost as xgb  # Import XGBoost
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Suppress Optuna's massive log output
optuna.logging.set_verbosity(optuna.logging.WARNING)

class ActivityModeler:
    def __init__(self, data, target_col='behavioral_category'):
        self.data = data
        self.target_col = target_col
        self.le = LabelEncoder()
        self.results_log = []
        
        if self.target_col not in self.data.columns:
            raise ValueError(f"Target column '{self.target_col}' not found.")
            
        counts = self.data[self.target_col].value_counts().to_dict()
        self.target_distribution_str = str(counts)

    def prepare_data(self, feature_cols):
        X = self.data[feature_cols]
        y = self.data[self.target_col]
        
        # XGBoost requires labels to be integers starting from 0
        y_encoded = self.le.fit_transform(y)
        
        # Stratify to maintain class balance
        try:
            X_train, X_test, y_train, y_test = train_test_split(
                X, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
            )
        except ValueError:
            # Fallback if a class has too few samples
            X_train, X_test, y_train, y_test = train_test_split(
                X, y_encoded, test_size=0.3, random_state=42
            )
        return X_train, X_test, y_train, y_test

    def get_optimizer_objective(self, trial, algo_name, X, y):
        """
        Defines the hyperparameter search space for each algorithm.
        Returns the cross-validation score to maximize.
        """
        # 1. XGBoost (New)
        if algo_name == "XGBoost":
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 300),
                'max_depth': trial.suggest_int('max_depth', 3, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'use_label_encoder': False,
                'eval_metric': 'mlogloss',
                'random_state': 42
            }
            model = xgb.XGBClassifier(**params)

        # 2. Random Forest
        elif algo_name == "RandomForest":
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 300),
                'max_depth': trial.suggest_int('max_depth', 5, 30),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
                'random_state': 42
            }
            model = RandomForestClassifier(**params)

        # 3. SVM
        elif algo_name == "SVM (RBF)":
            params = {
                'C': trial.suggest_float('C', 0.1, 100.0, log=True),
                'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),
                'kernel': 'rbf',
                'random_state': 42
            }
            model = SVC(**params)

        # 4. KNN
        elif algo_name == "KNN":
            params = {
                'n_neighbors': trial.suggest_int('n_neighbors', 3, 20),
                'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
                'p': trial.suggest_categorical('p', [1, 2])
            }
            model = KNeighborsClassifier(**params)

        # 5. Logistic Regression
        elif algo_name == "LogisticReg":
            params = {
                'C': trial.suggest_float('C', 0.01, 100.0, log=True),
                'solver': 'lbfgs',
                'max_iter': 2000,
                'random_state': 42
            }
            model = LogisticRegression(**params)
            
        else:
            raise ValueError(f"Unknown algorithm: {algo_name}")

        # Cross-Validation
        scores = cross_val_score(model, X, y, cv=3, scoring='f1_weighted')
        return scores.mean()

    def tune_and_evaluate(self, algo_name, X_train, X_test, y_train, y_test, n_trials=10):
        """
        Runs Optuna optimization, then trains the best model.
        """
        # Scaling is crucial for SVM/KNN/LogReg (XGBoost/RF handle unscaled fine, but scaling doesn't hurt)
        scaler = StandardScaler()
        X_train_s = scaler.fit_transform(X_train)
        X_test_s = scaler.transform(X_test)

        # --- 1. OPTUNA OPTIMIZATION ---
        def objective_wrapper(trial):
            return self.get_optimizer_objective(trial, algo_name, X_train_s, y_train)

        study = optuna.create_study(direction='maximize')
        study.optimize(objective_wrapper, n_trials=n_trials)
        
        best_params = study.best_params

        # --- 2. RE-TRAIN BEST MODEL ---
        if algo_name == "XGBoost":
            # Must ensure fixed params are passed again
            model = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
        elif algo_name == "RandomForest":
            model = RandomForestClassifier(**best_params, random_state=42)
        elif algo_name == "SVM (RBF)":
            model = SVC(**best_params, kernel='rbf', random_state=42)
        elif algo_name == "KNN":
            model = KNeighborsClassifier(**best_params)
        elif algo_name == "LogisticReg":
            model = LogisticRegression(**best_params, solver='lbfgs', max_iter=2000, random_state=42)

        model.fit(X_train_s, y_train)
        y_pred = model.predict(X_test_s)

        # --- 3. CALCULATE METRICS ---
        acc = accuracy_score(y_test, y_pred)
        prec, rec, f1, _ = precision_recall_fscore_support(
            y_test, y_pred, average='weighted', zero_division=0
        )

        return acc, prec, rec, f1, best_params

    def run_optuna_experiments(self, n_trials=10):
        """
        Main loop: Features -> Algorithms -> Optuna Tuning
        """
        base_features = [
            ("Raw Accel", ['x_g', 'y_g', 'z_g']),
            ("ODBA", ['odba']),
            ("VeDBA", ['vedba']),
            ("Magnitude", ['mag'])
        ]
        
        experiments_to_run = {}
        # Individual
        for name, cols in base_features:
            experiments_to_run[f"Indiv: {name}"] = cols
            
        # Cumulative
        curr_cols = []
        curr_names = []
        for name, cols in base_features:
            curr_cols = curr_cols + cols
            curr_names.append(name)
            seq_name = f"Seq: {' + '.join(curr_names)}"
            experiments_to_run[seq_name] = curr_cols

        # ADDED XGBOOST HERE
        algo_names = ["XGBoost", "RandomForest", "SVM (RBF)", "KNN", "LogisticReg"]
        
        print(f"Starting Optuna Tuning on {len(self.data)} rows.")
        print(f"Optimizing {len(experiments_to_run)} Feature Sets x {len(algo_names)} Models...")
        print("-" * 60)

        for feat_name, cols in experiments_to_run.items():
            if not set(cols).issubset(self.data.columns):
                print(f"⚠️ Skipping {feat_name} (Missing columns)")
                continue

            X_train, X_test, y_train, y_test = self.prepare_data(cols)

            for algo in algo_names:
                print(f" Tuning {algo} | Features: {feat_name}...")
                
                acc, prec, rec, f1, best_params = self.tune_and_evaluate(
                    algo, X_train, X_test, y_train, y_test, n_trials=n_trials
                )
                
                self.results_log.append({
                    'Algorithm': algo,
                    'Feature_Set': feat_name,
                    'Accuracy': round(acc, 4),
                    'Precision': round(prec, 4),
                    'Recall': round(rec, 4),
                    'F1_Score': round(f1, 4),
                    'Best_Params': str(best_params),
                    'Features_Used': str(cols)
                })

        return pd.DataFrame(self.results_log).sort_values(by='F1_Score', ascending=False)

In [11]:
import zipfile
import tempfile
import os
actual_file=[]
main_path="/home/rajesh/work/acclerometer_project/zip_data"
for zip_file in os.listdir(main_path):
        with tempfile.TemporaryDirectory() as temp_dir:
                zip_path= os.path.join(main_path, zip_file)
                with zipfile.ZipFile(zip_path, "r") as zf:
                        zf.extractall(temp_dir)
                        for root, dirs , files in os.walk(temp_dir):
                                for d in dirs:
                                        if d.startswith('Processed'):

                                                second_path= os.path.join(root, d)
                                                for excel_file in os.listdir(second_path):
                                                        actual_file.append(pd.read_excel(os.path.join(second_path, excel_file)))


In [12]:
data= pd.concat(actual_file)

In [19]:

df_raw = pd.DataFrame(data)

# 2. Run Pipeline
pipeline = AccelPipeline(df_raw)
pipeline.convert_to_gravity()
pipeline.calc_odba()
pipeline.calc_vedba()

# Resample (Note: with only 5 milliseconds of data, this might result in 1 row)
# For demo purposes, we resample to a very small window to force output, 
# in production use 5s or 10s.
df_ready = pipeline.resample_data(interval_seconds=5) 
 

# Initialize Modeler with correct target
modeler = ActivityModeler(df_ready, target_col='behavioral_category')

# Run all 4 feature sets


--- Converting to Gravity Units & Calculating ENMO ---
--- Calculating ODBA ---
--- Calculating VeDBA ---
--- Resampling data to 5 second windows ---


In [20]:
df=modeler.run_optuna_experiments()

Starting Optuna Tuning on 40488 rows.
Optimizing 8 Feature Sets x 5 Models...
------------------------------------------------------------
 Tuning XGBoost | Features: Indiv: Raw Accel...


/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [18:55:35] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [18:55:37] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [18:55:38] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [18:55:39] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/hom

 Tuning RandomForest | Features: Indiv: Raw Accel...
 Tuning SVM (RBF) | Features: Indiv: Raw Accel...
 Tuning KNN | Features: Indiv: Raw Accel...
 Tuning LogisticReg | Features: Indiv: Raw Accel...
 Tuning XGBoost | Features: Indiv: ODBA...


/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [19:03:58] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [19:04:00] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [19:04:03] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [19:04:07] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/hom

 Tuning RandomForest | Features: Indiv: ODBA...
 Tuning SVM (RBF) | Features: Indiv: ODBA...
 Tuning KNN | Features: Indiv: ODBA...
 Tuning LogisticReg | Features: Indiv: ODBA...
 Tuning XGBoost | Features: Indiv: VeDBA...


...

 Tuning RandomForest | Features: Seq: Raw Accel + ODBA...
 Tuning SVM (RBF) | Features: Seq: Raw Accel + ODBA...
 Tuning KNN | Features: Seq: Raw Accel + ODBA...
 Tuning LogisticReg | Features: Seq: Raw Accel + ODBA...
 Tuning XGBoost | Features: Seq: Raw Accel + ODBA + VeDBA...


/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [20:00:08] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [20:00:09] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [20:00:10] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [20:00:11] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/hom

 Tuning RandomForest | Features: Seq: Raw Accel + ODBA + VeDBA...
 Tuning SVM (RBF) | Features: Seq: Raw Accel + ODBA + VeDBA...
 Tuning KNN | Features: Seq: Raw Accel + ODBA + VeDBA...
 Tuning LogisticReg | Features: Seq: Raw Accel + ODBA + VeDBA...
 Tuning XGBoost | Features: Seq: Raw Accel + ODBA + VeDBA + Magnitude...


/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [20:13:14] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [20:13:15] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [20:13:17] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/rajesh/work/heat_stress_venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [20:13:19] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/hom

 Tuning RandomForest | Features: Seq: Raw Accel + ODBA + VeDBA + Magnitude...
 Tuning SVM (RBF) | Features: Seq: Raw Accel + ODBA + VeDBA + Magnitude...
 Tuning KNN | Features: Seq: Raw Accel + ODBA + VeDBA + Magnitude...
 Tuning LogisticReg | Features: Seq: Raw Accel + ODBA + VeDBA + Magnitude...


In [21]:
df

,Algorithm,Feature_Set,Accuracy,Precision,Recall,F1_Score,Best_Params,Features_Used
36,RandomForest,Seq: Raw Accel + ODBA + VeDBA + Magnitude,0.7403,0.7325,0.7403,0.7308,"{'n_estimators': 274, 'max_depth': 26, 'min_sa...","['x_g', 'y_g', 'z_g', 'odba', 'vedba', 'mag']"
35,XGBoost,Seq: Raw Accel + ODBA + VeDBA + Magnitude,0.7351,0.7272,0.7351,0.7270,"{'n_estimators': 277, 'max_depth': 8, 'learnin...","['x_g', 'y_g', 'z_g', 'odba', 'vedba', 'mag']"
38,KNN,Seq: Raw Accel + ODBA + VeDBA + Magnitude,0.7298,0.7201,0.7298,0.7206,"{'n_neighbors': 18, 'weights': 'distance', 'p'...","['x_g', 'y_g', 'z_g', 'odba', 'vedba', 'mag']"
37,SVM (RBF),Seq: Raw Accel + ODBA + VeDBA + Magnitude,0.7049,0.6962,0.7049,0.6912,"{'C': 34.04299060184746, 'gamma': 'scale'}","['x_g', 'y_g', 'z_g', 'odba', 'vedba', 'mag']"
26,RandomForest,Seq: Raw Accel + ODBA,0.7110,0.6903,0.7110,0.6902,"{'n_estimators': 152, 'max_depth': 22, 'min_sa...","['x_g', 'y_g', 'z_g', 'odba']"
31,RandomForest,Seq: Raw Accel + ODBA + VeDBA,0.7090,0.6854,0.7090,0.6859,"{'n_estimators': 149, 'max_depth': 29, 'min_sa...","['x_g', 'y_g', 'z_g', 'odba', 'vedba']"
28,KNN,Seq: Raw Accel + ODBA,0.6998,0.6729,0.6998,0.6771,"{'n_neighbors': 17, 'weights': 'distance', 'p'...","['x_g', 'y_g', 'z_g', 'odba']"
25,XGBoost,Seq: Raw Accel + ODBA,0.6993,0.6778,0.6993,0.6745,"{'n_estimators': 186, 'max_depth': 5, 'learnin...","['x_g', 'y_g', 'z_g', 'odba']"
30,XGBoost,Seq: Raw Accel + ODBA + VeDBA,0.6967,0.6711,0.6967,0.6721,"{'n_estimators': 106, 'max_depth': 6, 'learnin...","['x_g', 'y_g', 'z_g', 'odba', 'vedba']"
23,KNN,Seq: Raw Accel,0.6923,0.6644,0.6923,0.6720,"{'n_neighbors': 13, 'weights': 'distance', 'p'...","['x_g', 'y_g', 'z_g']"
